In [ ]:
# https://datahacker.rs/019-siamese-network-in-pytorch-with-application-to-face-similarity
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn 
import torchvision.models as models 
import torchvision.transforms as transforms
import os
import matplotlib.pyplot as plt
from glob import glob
from math import ceil, floor
import random 
from PIL import Image

: 